In [10]:
import pandas as pd
import numpy as np
import datetime as dt

In [ ]:
# Eu gostaria de transformar em função todo esse raciocínio que eu fiz, eu entraria com o ticker, com a data e
# ele retornaria o Book de Negociações bneg

In [15]:
def open_NEG_Book(file, index, ticker):
    f = open('../../data/b3/' + file,'r')
    i = pd.read_csv('../../data/b3/' + index, header = None, sep = ' ', index_col = 0, \
                names = ['Linicial', 'Lfinal'])
    
    Linicial = i.loc[ticker].iloc[0]['Linicial']
    Lfinal = i.loc[ticker].iloc[0]['Lfinal']
    
    f.seek(23)
    dia = f.read(10)

    start = 55
    linesize = 238

    data = []

    for line_n in range(Linicial, Lfinal+1):
        f.seek(start+line_n*linesize)
        line = f.readline()
        # Preço, Quantidade, Time
        data.append([ \
            line[74:74+19], \
            line[94:94+18], \
            dia + ' ' + line[113:113+12], \
            line[220:220+8], \
            line[229:229+8]
        ])
    
    bneg = pd.DataFrame(data, columns=['Price', 'Quant', 'Timestamp', 'CCompra', 'CVenda'])
    bneg['Price'] = bneg['Price'].astype(float)
    bneg['Quant'] = bneg['Quant'].astype(int)
    bneg['CCompra'] = bneg['CCompra'].astype(int)
    bneg['CVenda'] = bneg['CVenda'].astype(int)
    bneg['Timestamp'] = pd.to_datetime(bneg['Timestamp'], format='%Y-%m-%d %H:%M:%S.%f')
    bneg = bneg.set_index('Timestamp')
    
    cc = pd.read_csv('../../data/b3/Codigos_Corretoras.txt', header = None, sep = ';', \
                    names = ['Nome', 'Codigo'])
    cc['Codigo'] = cc['Codigo'].astype(int)
    cc = cc.set_index('Codigo')
    
    bneg = bneg.join(cc, on = 'CCompra')
    bneg = bneg.rename(columns={"Nome": "Corretora Compra"})
    bneg = bneg.join(cc, on = 'CVenda')
    bneg = bneg.rename(columns={"Nome": "Corretora Venda"})
    
    return bneg

In [17]:
bneg = open_NEG_Book('NEG_BMF_20191126.TXT', 'NEG_BMF_20191126_index.TXT', 'INDZ19')

In [89]:
# aqui eu salvo um arquivo de log para analisar com mais calma
bneg.to_csv('book.txt', sep = '\t', mode = 'w')